In [ ]:
# Load and preprocess the dataset (e.g., MNIST)
from tensorflow.keras.datasets import mnist
import numpy as np

(x_train, _), (x_test, _) = mnist.load_data()

# Normalize the data
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

# Reshape the data to match the expected input shape of the model
x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))
x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))

# Add random noise to the training data
noise_factor = 0.2
x_train_noisy = x_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_train.shape)
x_test_noisy = x_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_test.shape)

# Clip the values to ensure they are in the range [0, 1]
x_train_noisy = np.clip(x_train_noisy, 0., 1.)
x_test_noisy = np.clip(x_test_noisy, 0., 1.)

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D

# Define the input shape
input_shape = (28, 28, 1)

# Encoder
inputs = Input(shape=input_shape)
conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
pool1 = MaxPooling2D((2, 2), padding='same')(conv1)
conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
pool2 = MaxPooling2D((2, 2), padding='same')(conv2)
encoded = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)

# Decoder
conv3 = Conv2D(64, (3, 3), activation='relu', padding='same')(encoded)
up1 = UpSampling2D((2, 2))(conv3)
conv4 = Conv2D(32, (3, 3), activation='relu', padding='same')(up1)
up2 = UpSampling2D((2, 2))(conv4)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(up2)

# Define the autoencoder model
autoencoder = Model(inputs, decoded)

from tensorflow.keras.optimizers import Adam

# Compile the model
autoencoder.compile(optimizer=Adam(), loss='binary_crossentropy')

# Train the model
epochs = 50
batch_size = 128
autoencoder.fit(x_train_noisy, x_train,
                epochs=epochs,
                batch_size=batch_size,
                shuffle=True,
                validation_data=(x_test_noisy, x_test))

# Evaluate the model on the test set
decoded_imgs = autoencoder.predict(x_test_noisy)

# Calculate the mean squared error
import tensorflow.keras.backend as K

mse = K.mean(K.square(x_test - decoded_imgs))
print('Mean Squared Error:', mse)

# Visualize the reconstructed images
import matplotlib.pyplot as plt

n = 10
plt.figure(figsize=(20, 4))
for i in range(n):
    # Display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

plt.show()

from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

# Define the input shape
input_shape = (28, 28, 1)

# Define the autoencoder model
def create_autoencoder(learning_rate=0.001, loss='binary_crossentropy'):
    inputs = Input(shape=input_shape)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    pool1 = MaxPooling2D((2, 2), padding='same')(conv1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    pool2 = MaxPooling2D((2, 2), padding='same')(conv2)
    encoded = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)

    conv3 = Conv2D(64, (3, 3), activation='relu', padding='same')(encoded)
    up1 = UpSampling2D((2, 2))(conv3)
    conv4 = Conv2D(32, (3, 3), activation='relu', padding='same')(up1)
    up2 = UpSampling2D((2, 2))(conv4)
    decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(up2)

    autoencoder = Model(inputs, decoded)
    optimizer = Adam(learning_rate=learning_rate)
    autoencoder.compile(optimizer=optimizer, loss=loss)
    return autoencoder

# Create a KerasClassifier
autoencoder_clf = KerasClassifier(build_fn=create_autoencoder, verbose=0)

# Define the hyperparameters to tune
learning_rates = [0.001, 0.01, 0.1]
losses = ['binary_crossentropy', 'mean_squared_error']
param_grid = dict(learning_rate=learning_rates, loss=losses)

# Perform grid search
grid_search = GridSearchCV(estimator=autoencoder_clf, param_grid=param_grid, n_jobs=-1, cv=3)
grid_search = grid_search.fit(x_train_noisy, x_train)

# Print the best hyperparameters
print("Best hyperparameters: ", grid_search.best_params_)

# Get the best model
best_model = grid_search.best_estimator_.model

# Evaluate the best model on the test set
decoded_imgs = best_model.predict(x_test_noisy)

# Calculate the mean squared error
import tensorflow.keras.backend as K

mse = K.mean(K.square(x_test - decoded_imgs))
print('Mean Squared Error:', mse)

# Visualize the reconstructed images
import matplotlib.pyplot as plt

n = 10
plt.figure(figsize=(20, 4))
for i in range(n):
    # Display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

plt.show()

Epoch 1/50
 58/469 [==>...........................] - ETA: 3:35 - loss: 0.2794

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import GridSearchCV

# Load the IMDb dataset
max_features = 10000
maxlen = 200
(X_train_txt, y_train_txt), (X_test_txt, y_test_txt) = imdb.load_data(num_words=max_features)

# Preprocess the data
X_train_txt = pad_sequences(X_train_txt, maxlen=maxlen)
X_test_txt = pad_sequences(X_test_txt, maxlen=maxlen)

# Build RNN model
def create_rnn_model(optimizer='adam'):
    model = Sequential([
        Embedding(max_features, 32),
        SimpleRNN(32),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Perform grid search for RNN model hyperparameters
rnn_model = create_rnn_model()
rnn_param_grid = {'optimizer': ['adam', 'rmsprop'],
                  'batch_size': [32, 64],
                  'learning_rate': [0.01, 0.001]}
rnn_grid = GridSearchCV(estimator=rnn_model, param_grid=rnn_param_grid, cv=3)
rnn_grid_result = rnn_grid.fit(X_train_txt, y_train_txt)

# Print RNN grid search results
print("Best RNN Model: %f using %s" % (rnn_grid_result.best_score_, rnn_grid_result.best_params_))

# Train the final RNN model with best hyperparameters
best_rnn_params = rnn_grid_result.best_params_
final_rnn_model = create_rnn_model(optimizer=best_rnn_params['optimizer'])
rnn_history = final_rnn_model.fit(X_train_txt, y_train_txt, epochs=5, batch_size=best_rnn_params['batch_size'], validation_split=0.2)

# Evaluate RNN model on test set
rnn_test_loss, rnn_test_acc = final_rnn_model.evaluate(X_test_txt, y_test_txt)
print('RNN Test Accuracy:', rnn_test_acc)

# Plot RNN model training history
plt.plot(rnn_history.history['accuracy'], label='accuracy')
plt.plot(rnn_history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

